## Load libraries

In [1]:
# Libraries to work cross-platform
import os

# Libraries to work with dataset
import numpy as np
import pandas as pd
import ast                      # convert string to list after importing csv data
import tmtoolkit
import pickle

# Libraries to cluster data
from gsdmm import MovieGroupProcess

# Libraries to visualize data
# import matplotlib.pyplot as plt
import seaborn as sns
from cluster_visualization_helper import (
    visualize_cluster  # user-defined functions
)

# Libraries for evaluation
from sklearn import metrics

# Libraries for monitoring operation process
from tqdm import tqdm
from datetime import datetime
# from pprint import pprint
# import logging

## Configurate and declare global variables

In [2]:
os_name = os.name

if os_name == 'nt':
    BASE_DIR = "E:/THIENDHB_GOOGLEDRIVE/MASTER TILBURG/THESIS/"
    INPUT_DIR = BASE_DIR + "DATASET/INPUT/"
    OUTPUT_DIR = BASE_DIR + "DATASET/OUTPUT/"
elif os_name == 'posix':
    BASE_DIR = "/media/pinkalinux/WORK/THIENDHB_GOOGLEDRIVE/MASTER TILBURG/THESIS/"
    INPUT_DIR = BASE_DIR + "DATASET/INPUT/"
    OUTPUT_DIR = BASE_DIR + "DATASET/OUTPUT/"

SEED = 6886

%matplotlib inline

## Import data

In [3]:
title_embeddings = np.load(OUTPUT_DIR + "title_embeddings.npy")

In [4]:
# title_embeddings.shape

In [5]:
title_docs = pd.read_csv(OUTPUT_DIR + "title_docs.csv")

In [6]:
title_docs["clean_job_title"] = title_docs["clean_job_title"].apply(ast.literal_eval)

## Clustering data

### Gibbs Sampling Dirichlet Mixture Model (GSDMM)

In [7]:
title_corpus = title_docs["clean_job_title"].values.tolist()
title_vocab = set(tmtoolkit.utils.flatten_list(title_corpus))
len(title_vocab), list(title_vocab)[:10]

(2617,
 ['premium',
  'replacement',
  'registry',
  'update',
  'course',
  'road',
  'seg',
  'introduce',
  'speaker',
  'landmine'])

In [8]:
def build_mgp_model(
    X,
    corpus,
    vocab_size,
    K=100,
    alpha=0.1,
    beta=0.1,
    n_iters=50,
    seed=6886,
):
    """
    A MovieGroupProcess is a conceptual model introduced by Yin and Wang 2014 to
    describe their Gibbs sampling algorithm for a Dirichlet Mixture Model for the
    clustering short text documents.
    Reference: http://dbgroup.cs.tsinghua.edu.cn/wangjy/papers/KDD14-GSDMM.pdf
    Imagine a professor is leading a film class. At the start of the class, the students
    are randomly assigned to K tables. Before class begins, the students make lists of
    their favorite films. The teacher reads the role n_iters times. When
    a student is called, the student must select a new table satisfying either:
        1) The new table has more students than the current table.
    OR
        2) The new table has students with similar lists of favorite movies.

    :param K: int
        Upper bound on the number of possible clusters. Typically many fewer
    :param alpha: float between 0 and 1
        Alpha controls the probability that a student will join a table that is currently empty
        When alpha is 0, no one will join an empty table.
    :param beta: float between 0 and 1
        Beta controls the student's affinity for other students with similar interests. A low beta means
        that students desire to sit with students of similar interests. A high beta means they are less
        concerned with affinity and are more influenced by the popularity of a table
    """

#     logging.debug("Start of GSDMM model for job titles")

    np.random.seed(SEED)
    mgp = MovieGroupProcess(K=K, alpha=alpha, beta=beta, n_iters=n_iters)

    """
        MovieGroupProcess().fit: Cluster the input documents
        :param docs: list of list
            list of lists containing the unique token set of each document
        :param V: total vocabulary size for each document
        :return: list of length len(doc)
            cluster label for each document
    """
    mgp_labels = mgp.fit(docs=corpus, vocab_size=vocab_size)

#     logging.debug("End of GSDMM model for job titles")

    return mgp, mgp_labels

In [9]:
# Define search parameters
k_range = [50, 100, 150]
alpha_range = [0.01, 0.25, 0.5, 0.75, 1.0]
beta_range = [0.01, 0.25, 0.5, 0.75, 1.0]
n_iter_range = [50, 100]
# k_range = [5]
# alpha_range = [0.5]
# beta_range = [0.5]
# n_iter_range = [5]

print(
    "Total hyperparameters in search space =",
    str(len(k_range) * len(alpha_range) * len(beta_range) * len(n_iter_range)),
)

Total hyperparameters in search space = 150


In [10]:
# # Set up logger for callback

# # Creating logger
# mylogs = logging.getLogger(__name__)
# mylogs.setLevel(logging.NOTSET)

# # Handler - log in file
# log_file = logging.FileHandler(OUTPUT_DIR + "title_gsdmm_model_callbacks.log")
# fileformat = logging.Formatter("%(asctime)s:%(levelname)s:%(message)s")
# log_file.setLevel(logging.NOTSET)
# log_file.setFormatter(fileformat)

# # Handler - log in console
# # stream = logging.StreamHandler()
# # streamformat = logging.Formatter("%(levelname)s:%(module)s:%(message)s")
# # stream.setLevel(logging.NOTSET)
# # stream.setFormatter(streamformat)

# # Adding all handlers to the logs
# mylogs.addHandler(log_file)
# # mylogs.addHandler(stream)
# print = mylogs.info

In [ ]:
# # Set up logger for callback
# logging.basicConfig(
# #     filename=OUTPUT_DIR + "skills_lda_model_callbacks.log",
#     handlers=[
#         logging.FileHandler(
#             OUTPUT_DIR + "title_gsdmm_model_callbacks.log", "w", "utf-8"
#         )
#     ],
#     format="%(asctime)s:%(levelname)s:%(message)s",
#     level=logging.NOTSET,
# )

# Create empty list to store labels created by models
labels_list = []
model_list = []
duration_list = []
model_result = {"K": [], "alpha": [], "beta": [], "n_iters": [], "Silhouette": []}

start_loop_time = datetime.now()
print(f"Start loop {start_loop_time.strftime('%Y-%m-%d %H:%M:%S.%f')}")

# Loop through the search space
tqdm_bar = tqdm(
    desc="Tuning GSDMM model",
    total=len(k_range) * len(alpha_range) * len(beta_range) * len(n_iter_range),
)

for K in k_range:
    for alpha in alpha_range:
        for beta in beta_range:
            for n_iters in n_iter_range:
                print(f"K = {K}")
                print(f"alpha = {alpha}")
                print(f"beta = {beta}")
                print(f"n_iters = {n_iters}")

                starttime = datetime.now()
                print(f"Start {starttime.strftime('%Y-%m-%d %H:%M:%S.%f')}")

                mgp_model, mgp_labels = build_mgp_model(
                    X=title_docs,
                    corpus=title_corpus,
                    vocab_size=len(title_vocab),
                    K=K,
                    alpha=alpha,
                    beta=beta,
                    n_iters=n_iters,
                    seed=SEED,
                )

                endtime = datetime.now()
                print(f"End {endtime.strftime('%Y-%m-%d %H:%M:%S.%f')}")
                print(f"Duration {endtime - starttime}")

                duration_list.append(round((endtime - starttime).seconds / 60, 2))
                labels_list.append(mgp_labels)
                model_list.append(mgp_model)

                model_result["K"].append(K)
                model_result["alpha"].append(alpha)
                model_result["beta"].append(beta)
                model_result["n_iters"].append(n_iters)

                tqdm_bar.update(1)

tqdm_bar.close()

end_loop_time = datetime.now()
print(f"End loop {end_loop_time.strftime('%Y-%m-%d %H:%M:%S.%f')}")
print(f"Duration {end_loop_time - start_loop_time}")

Tuning GSDMM model:   0%|                                                                                       | 0/150 [00:00<?, ?it/s]

Start loop 2021-05-04 02:27:01.723086
K = 50
alpha = 0.01
beta = 0.01
n_iters = 50
Start 2021-05-04 02:27:01.762087
In stage 0: transferred 17800 clusters with 50 clusters populated
In stage 1: transferred 13725 clusters with 50 clusters populated
In stage 2: transferred 9785 clusters with 50 clusters populated
In stage 3: transferred 7733 clusters with 50 clusters populated
In stage 4: transferred 6614 clusters with 50 clusters populated
In stage 5: transferred 6051 clusters with 50 clusters populated
In stage 6: transferred 5645 clusters with 50 clusters populated
In stage 7: transferred 5502 clusters with 50 clusters populated
In stage 8: transferred 5408 clusters with 50 clusters populated
In stage 9: transferred 5341 clusters with 50 clusters populated
In stage 10: transferred 5269 clusters with 50 clusters populated
In stage 11: transferred 5126 clusters with 50 clusters populated
In stage 12: transferred 5048 clusters with 50 clusters populated
In stage 13: transferred 4941 clus

Tuning GSDMM model:   1%|▌                                                                          | 1/150 [10:37<26:24:14, 637.95s/it]

In stage 49: transferred 4316 clusters with 50 clusters populated
End 2021-05-04 02:37:39.707631
Duration 0:10:37.945544
K = 50
alpha = 0.01
beta = 0.01
n_iters = 100
Start 2021-05-04 02:37:39.708631
In stage 0: transferred 17800 clusters with 50 clusters populated
In stage 1: transferred 13725 clusters with 50 clusters populated
In stage 2: transferred 9785 clusters with 50 clusters populated
In stage 3: transferred 7733 clusters with 50 clusters populated
In stage 4: transferred 6614 clusters with 50 clusters populated
In stage 5: transferred 6051 clusters with 50 clusters populated
In stage 6: transferred 5645 clusters with 50 clusters populated
In stage 7: transferred 5502 clusters with 50 clusters populated
In stage 8: transferred 5408 clusters with 50 clusters populated
In stage 9: transferred 5341 clusters with 50 clusters populated
In stage 10: transferred 5269 clusters with 50 clusters populated
In stage 11: transferred 5126 clusters with 50 clusters populated
In stage 12: tra

Tuning GSDMM model:   1%|█                                                                          | 2/150 [30:31<39:40:17, 964.98s/it]

...

In stage 49: transferred 4277 clusters with 50 clusters populated
End 2021-05-04 09:38:19.445551
Duration 0:06:46.041191
K = 50
alpha = 1.0
beta = 0.01
n_iters = 100
Start 2021-05-04 09:38:19.447537
In stage 0: transferred 17799 clusters with 50 clusters populated
In stage 1: transferred 13752 clusters with 50 clusters populated
In stage 2: transferred 10071 clusters with 50 clusters populated
In stage 3: transferred 7982 clusters with 50 clusters populated
In stage 4: transferred 6920 clusters with 50 clusters populated
In stage 5: transferred 6170 clusters with 50 clusters populated
In stage 6: transferred 5703 clusters with 50 clusters populated
In stage 7: transferred 5579 clusters with 50 clusters populated
In stage 8: transferred 5339 clusters with 50 clusters populated
In stage 9: transferred 5257 clusters with 50 clusters populated
In stage 10: transferred 5083 clusters with 50 clusters populated
In stage 11: transferred 5046 clusters with 50 clusters populated
In stage 12: tra

Tuning GSDMM model:  28%|████████████████████▏                                                   | 42/150 [7:24:59<19:23:06, 646.18s/it]

In stage 99: transferred 4230 clusters with 50 clusters populated
End 2021-05-04 09:52:00.843926
Duration 0:13:41.396389
K = 50
alpha = 1.0
beta = 0.25
n_iters = 50
Start 2021-05-04 09:52:00.845924
In stage 0: transferred 18278 clusters with 50 clusters populated
In stage 1: transferred 14842 clusters with 50 clusters populated
In stage 2: transferred 9958 clusters with 50 clusters populated
In stage 3: transferred 7111 clusters with 50 clusters populated
In stage 4: transferred 5763 clusters with 47 clusters populated
In stage 5: transferred 4946 clusters with 48 clusters populated
In stage 6: transferred 4620 clusters with 47 clusters populated
In stage 7: transferred 4453 clusters with 46 clusters populated
In stage 8: transferred 4391 clusters with 43 clusters populated
In stage 9: transferred 4236 clusters with 42 clusters populated
In stage 10: transferred 4234 clusters with 38 clusters populated
In stage 11: transferred 4258 clusters with 38 clusters populated
In stage 12: trans

Tuning GSDMM model:  29%|████████████████████▋                                                   | 43/150 [7:31:49<17:06:28, 575.59s/it]

In stage 49: transferred 3761 clusters with 30 clusters populated
End 2021-05-04 09:58:51.736251
Duration 0:06:50.890327
K = 50
alpha = 1.0
beta = 0.25
n_iters = 100
Start 2021-05-04 09:58:51.736251
In stage 0: transferred 18278 clusters with 50 clusters populated
In stage 1: transferred 14842 clusters with 50 clusters populated
In stage 2: transferred 9958 clusters with 50 clusters populated
In stage 3: transferred 7111 clusters with 50 clusters populated
In stage 4: transferred 5763 clusters with 47 clusters populated
In stage 5: transferred 4946 clusters with 48 clusters populated
In stage 6: transferred 4620 clusters with 47 clusters populated
In stage 7: transferred 4453 clusters with 46 clusters populated
In stage 8: transferred 4391 clusters with 43 clusters populated
In stage 9: transferred 4236 clusters with 42 clusters populated
In stage 10: transferred 4234 clusters with 38 clusters populated
In stage 11: transferred 4258 clusters with 38 clusters populated
In stage 12: tran

In [ ]:
# np.vstack(
#     (
#         np.unique(gsdmm_cluster_labels, return_counts=True)[0],
#         np.unique(gsdmm_cluster_labels, return_counts=True)[1],
#     )
# ).T

## Tuning hyperparameters

In [ ]:
# # Plot the log-likelihood
# plt.figure(figsize=(12, 8))
# plt.plot(n_range, score_list, "-o", color="brown")
# plt.xlabel("Number of Components")
# plt.ylabel("Log-likelihood")
# plt.xticks(n_range)
# plt.title("Average log-likelihood of BGM models")
# plt.grid()

In [ ]:
# i = 0
# silho_table = {"K": [], "alpha": [], "beta": [], "n_iters": [], "Silhouette": []}

# for K in k_range:
#     #     print('K =', K)
#     for alpha in alpha_range:
#         #         print('alpha =', alpha)
#         for beta in beta_range:
#             #             print('beta =', beta)
#             for n_iters in n_iter_range:
#                 #                 print('n_iters =', n_iters)
#                 #                 print("i =", i)
#                 silho_score = metrics.silhouette_score(title_embeddings, labels_list[i])
#                 silho_table["K"].append(K)
#                 silho_table["alpha"].append(alpha)
#                 silho_table["beta"].append(beta)
#                 silho_table["n_iters"].append(n_iters)
#                 silho_table["Silhouette"].append(silho_score)
#                 i += 1

# # pd.DataFrame(silho_table)

In [ ]:
# Calculate Silhouette score
for idx, _ in tqdm(enumerate(model_list)):
    silho_score = metrics.silhouette_score(title_embeddings, labels_list[idx])
    model_result["Silhouette"].append(silho_score)

In [ ]:
def get_average_score(models, docs):
    avg_score_list = []
    for model in models:
        score_list = []
        for doc in docs:
            score = model.score(doc)
            score_list.append(score)
#             print(score_list)
            avg_score = np.mean(score_list)
#             print(avg_score)
        avg_score_list.append(avg_score)
    return avg_score_list

In [ ]:
# Create average score
avg_scores = get_average_score(model_list, title_docs)
# print(avg_scores)
model_result['Average Score'] = avg_scores

In [ ]:
pd.DataFrame(model_result)
# model_result

In [ ]:
# Save tuning results to file
pd.DataFrame(model_result).to_csv(
    OUTPUT_DIR + "job-title_gsdmm-tuning-results.csv", index=False
)

In [ ]:
# Combine model sets
tuple_objects = (
    k_range,
    alpha_range,
    beta_range,
    model_list,
    duration_list,
    labels_list,
    model_result
)
len(tuple_objects), len(tuple_objects[0])

In [ ]:
# Save tuple
pickle.dump(tuple_objects, open(OUTPUT_DIR + "title_gsdmm_tuning_results.pkl", "wb"))

In [ ]:
# Select best model base on Silhouette score
best_index = np.argmax(model_result["Silhouette"])
# best_score = silho_table["Silhouette"][best_index]
best_n = len(np.unique(labels_list[best_index]))
best_model = model_list[best_index]
best_labels = labels_list[best_index]
best_params = [(key, value[best_index]) for key, value in model_result.items()]
best_index, best_n, best_params

In [ ]:
# # indexes = len(silho_list) % (len(alpha_range)*len(beta_range)*len(n_iter_range))
# score_range = np.arange(len(silho_list))
# print(score_range)

# indexes = score_range % (len(alpha_range)*len(beta_range)*len(n_iter_range))
# print(indexes)

# print(score_range[indexes == 0])
# print(silho_list)
# print(np.asarray(silho_list)[np.where(np.asarray(indexes) == 0)])

In [ ]:
# silho_df = pd.DataFrame(silho_table)
# # Plot the Silhouette score
# for b in beta_range:
#     plt.figure(figsize=(16, 5))
#     for ids, a in enumerate(alpha_range):
#         plot_df = silho_df.loc[silho_df["beta"] == b]
#         plot_df = plot_df.loc[plot_df['alpha'] == a]
#         plt.subplot(1, len(alpha_range), ids+1)
#         for n_iters in n_iter_range:
#             plt.plot(
#                 plot_df["K"].loc[plot_df['n_iters'] == n_iters],
#                 plot_df["Silhouette"].loc[plot_df['n_iters'] == n_iters],
#                 label=f"n_iters = {n_iters}",
#             )
#         plt.title(f"Alpha = {a} \n Beta = {b}")
#         plt.legend(loc="best")
#         plt.xlabel("Number of maximum usable clusters")
#         plt.xticks(k_range)
#         plt.ylabel("Silhouette score")
#         plt.grid()
#     plt.show()

In [ ]:
pca_datapoint = np.load(OUTPUT_DIR + "pca_datapoints.npy")
tsne_datapoint = np.load(OUTPUT_DIR + "tsne_datapoints.npy")

In [ ]:
plot_title = f"GSDMM visualization of job titles (number of components = {best_n})"
palette = sns.color_palette("hls", as_cmap=True)
colors = np.asarray(best_labels) / best_n

# Visualize clusters with PCA and t-SNE
visualize_cluster(
    plot_title,
    (12, 12),
    colors,
    palette,
    pca_datapoint,
    tsne_datapoint,
    None,
    pca=True,
    tsne=True,
    mds=False
)

## Save cluster results to file

In [ ]:
np.save(OUTPUT_DIR + "job-title_gsdmm-clusters.csv", best_labels)